# Suffix Tree

String is used to label the edges, instead of (offset, length), to keep the code simpler. However, (offset, length) should be used instead in practice to have a more efficient storage space.

In [94]:
class SuffixTree(object):
    
    class Node(object):
        def __init__(self, lab):
            self.lab = lab # label on path leading to this node
            self.out = {}  # outgoing edges; maps characters to nodes
    
    def __init__(self, s):
        """ Make suffix tree, without suffix links, from s in quadratic time
            and linear space """
        s += '$'
        self.root = self.Node(None)
        self.root.out[s[0]] = self.Node(s) # trie for just longest suf
        # add the rest of the suffixes, from longest to shortest
        for i in range(1, len(s)):
            # start at root; we’ll walk down as far as we can go
            cur = self.root
            j = i
            while j < len(s):
                if s[j] in cur.out:
                    child = cur.out[s[j]]
                    lab = child.lab
                    # Walk along edge until we exhaust edge label or
                    # until we mismatch
                    k = j+1 
                    while k-j < len(lab) and s[k] == lab[k-j]:
                        k += 1
                    if k-j == len(lab):
                        cur = child # we exhausted the edge
                        j = k
                    else:
                        # we fell off in middle of edge
                        cExist, cNew = lab[k-j], s[k]
                        # create “mid”: new node bisecting edge
                        mid = self.Node(lab[:k-j])
                        mid.out[cNew] = self.Node(s[k:])
                        # original child becomes mid’s child
                        mid.out[cExist] = child
                        # original child’s label is curtailed
                        child.lab = lab[k-j:]
                        # mid becomes new child of original parent
                        cur.out[s[j]] = mid
                else:
                    # Fell off tree at a node: make new edge hanging off it
                    cur.out[s[j]] = self.Node(s[j:])
        # Store the input text
        self.text = s[:-1]  # Remove the last character '$'
    
    def followPath(self, s):
        """ Follow path given by s.  If we fall off tree, return None.  If we
            finish mid-edge, return (node, offset) where 'node' is child and
            'offset' is label offset.  If we finish on a node, return (node,
            None). """
        cur = self.root
        i = 0
        while i < len(s):
            c = s[i]
            if c not in cur.out:
                return (None, None) # fell off at a node
            child = cur.out[s[i]]
            lab = child.lab
            j = i+1
            while j-i < len(lab) and j < len(s) and s[j] == lab[j-i]:
                j += 1
            if j-i == len(lab):
                cur = child # exhausted edge
                i = j
            elif j == len(s):
                return (child, j-i) # exhausted query string in middle of edge
            else:
                return (None, None) # fell off in the middle of the edge
        return (cur, None) # exhausted query string at internal node
    
    def hasSubstring(self, s):
        """ Return true iff s appears as a substring """
        node, off = self.followPath(s)
        return node is not None
    
    def searchSubstring(self, substring):
        cur = self.root
        i = 0
        while i < len(substring):
            c = substring[i]        
            if c not in cur.out:
                return False  # fell off the tree
            child = cur.out[c]
            lab = child.lab
            j = 0
            while i < len(substring) and j < len(lab) and substring[i] == lab[j]:
                i += 1
                j += 1
            if j == len(lab):
                cur = child  # exhausted edge
            else:
                return False  # mismatch
        
        # If the traversal reached the end of the substring, it means the substring is found
        return True
    
    def hasSuffix(self, s):
        """ Return true iff s is a suffix """
        node, off = self.followPath(s)
        if node is None:
            return False # fell off the tree
        if off is None:
            # finished on top of a node
            return '$' in node.out
        else:
            # finished at offset 'off' within an edge leading to 'node'
            return node.lab[off] == '$'

    def setText(self, text):
        self.text = text

    def countSubstring(self, substring):
        """ Return the number of times substring occurs in the text """
        count = 0
        start = 0
        while start < len(self.text):
            pos = self.text.find(substring, start)
            if pos == -1:
                break
            count += 1
            start = pos + 1
        return count
    
    def findSubstringPositions(self, substring):
        """ Print and return a list of indices where substring occurs in the text """
        positions = []
        start = 0
        while start < len(self.text):
            pos = self.text.find(substring, start)
            if pos == -1:
                break
            positions.append(pos)
            print(f"'{substring}' found at position {pos}")
            start = pos + 1
        return positions

In [95]:
princeTree = SuffixTree('For these reasons Louis the Twelfth, King of France, quickly occupied Milan, and as quickly lost it; and to turn him out the first time it only needed Lodovico’s own forces; because those who had opened the gates to him, finding themselves deceived in their hopes of future benefit, would not endure the ill-treatment of the new prince. It is very true that, after acquiring rebellious provinces a second time, they are not so lightly lost afterwards, because the prince, with little reluctance, takes the opportunity of the rebellion to punish the delinquents, to clear out the suspects, and to strengthen himself in the weakest places. Thus to cause France to lose Milan the first time it was enough for the Duke Lodovico[1] to raise insurrections on the borders; but to cause him to lose it a second time it was necessary to bring the whole world against him, and that his armies should be defeated and driven out of Italy; which followed from the causes above mentioned')

In [96]:
princeTree.hasSubstring('Milan')

True

In [97]:
princeTree.hasSubstring('France')

True

In [98]:
princeTree.hasSubstring('against him')

True

In [99]:
princeTree.hasSubstring('Berlin')

True

In [100]:
princeTree.hasSubstring('Duke')

True

In [101]:
princeTree.countSubstring("Milan")

2

In [102]:
princeTree.countSubstring('against him')

1

In [103]:
princeTree.countSubstring('Berlin')

0

In [104]:
princeTree.findSubstringPositions('Milan')

'Milan' found at position 70
'Milan' found at position 667


[70, 667]

In [105]:
princeTree.findSubstringPositions('France')

'France' found at position 45
'France' found at position 652


[45, 652]

In [12]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

lambdaPhageVirus = readGenome('GCA_000840245.1_ViralProj14204_genomic.fna')
lambdaPhageVirus[:100]

'GGGCGGCGACCTCGCGGGTTTTCGCTATTTATGAAAATTTTCCGGTTTAAGGCGTTTCCGTTCTTCTTCGTCATAACTTAATGTTTTTATTTAAAATACC'

In [106]:
lambdaPhageVirusSuffixTree = SuffixTree(lambdaPhageVirus)

KeyboardInterrupt: 

In [107]:
lambdaPhageVirusSuffixTree.hasSubstring('TACC')

True

In [108]:
lambdaPhageVirusSuffixTree.hasSubstring('TCCGGTTTAAGGCGTTTCCGTTC')

False

In [109]:
lambdaPhageVirusSuffixTree.hasSubstring('TCCGGTTTAAGGCGAAAACGTTC')

False

In [110]:
lambdaPhageVirusSuffixTree.countSubstring('TACC')

147

In [111]:
lambdaPhageVirusSuffixTree.findSubstringPositions('TACC')

'TACC' found at position 96
'TACC' found at position 240
'TACC' found at position 695
'TACC' found at position 845
'TACC' found at position 1208
'TACC' found at position 1892
'TACC' found at position 2056
'TACC' found at position 2243
'TACC' found at position 2248
'TACC' found at position 2266
'TACC' found at position 2993
'TACC' found at position 3236
'TACC' found at position 3308
'TACC' found at position 3380
'TACC' found at position 3499
'TACC' found at position 3511
'TACC' found at position 3823
'TACC' found at position 4114
'TACC' found at position 4253
'TACC' found at position 5175
'TACC' found at position 5205
'TACC' found at position 5766
'TACC' found at position 5773
'TACC' found at position 5805
'TACC' found at position 6082
'TACC' found at position 6449
'TACC' found at position 6790
'TACC' found at position 7043
'TACC' found at position 7419
'TACC' found at position 7936
'TACC' found at position 8408
'TACC' found at position 8718
'TACC' found at position 8875
'TACC' found at

[96,
 240,
 695,
 845,
 1208,
 1892,
 2056,
 2243,
 2248,
 2266,
 2993,
 3236,
 3308,
 3380,
 3499,
 3511,
 3823,
 4114,
 4253,
 5175,
 5205,
 5766,
 5773,
 5805,
 6082,
 6449,
 6790,
 7043,
 7419,
 7936,
 8408,
 8718,
 8875,
 8931,
 8961,
 9190,
 9265,
 9676,
 9976,
 10022,
 10236,
 11497,
 11707,
 11731,
 11785,
 12854,
 15122,
 15530,
 15677,
 15845,
 16137,
 16485,
 16817,
 16924,
 17054,
 17535,
 17943,
 18317,
 18557,
 19110,
 19147,
 19375,
 19449,
 20898,
 21021,
 21110,
 21208,
 21325,
 22076,
 22097,
 22249,
 22262,
 22865,
 23186,
 23545,
 23733,
 24244,
 24280,
 24419,
 24734,
 24862,
 24919,
 25185,
 25335,
 25406,
 26180,
 26372,
 26562,
 26938,
 27184,
 27598,
 28019,
 28357,
 28785,
 28934,
 29540,
 30299,
 30786,
 31007,
 31276,
 31321,
 31516,
 32305,
 32818,
 32867,
 33612,
 34109,
 34435,
 35112,
 35276,
 36007,
 36020,
 36637,
 37495,
 37547,
 37997,
 38926,
 39928,
 40051,
 40306,
 40450,
 40461,
 40870,
 41249,
 41618,
 41779,
 41877,
 42061,
 42067,
 42140,
 427

In [117]:
lambdaPhageVirusSuffixTree.findSubstringPositions('GGG')

'GGG' found at position 0
'GGG' found at position 15
'GGG' found at position 264
'GGG' found at position 294
'GGG' found at position 347
'GGG' found at position 380
'GGG' found at position 580
'GGG' found at position 647
'GGG' found at position 699
'GGG' found at position 700
'GGG' found at position 761
'GGG' found at position 807
'GGG' found at position 854
'GGG' found at position 861
'GGG' found at position 882
'GGG' found at position 901
'GGG' found at position 970
'GGG' found at position 1131
'GGG' found at position 1166
'GGG' found at position 1186
'GGG' found at position 1234
'GGG' found at position 1297
'GGG' found at position 1439
'GGG' found at position 1440
'GGG' found at position 1604
'GGG' found at position 1716
'GGG' found at position 1751
'GGG' found at position 1752
'GGG' found at position 1759
'GGG' found at position 1803
'GGG' found at position 1944
'GGG' found at position 1945
'GGG' found at position 1950
'GGG' found at position 1951
'GGG' found at position 1952
'GGG'

[0,
 15,
 264,
 294,
 347,
 380,
 580,
 647,
 699,
 700,
 761,
 807,
 854,
 861,
 882,
 901,
 970,
 1131,
 1166,
 1186,
 1234,
 1297,
 1439,
 1440,
 1604,
 1716,
 1751,
 1752,
 1759,
 1803,
 1944,
 1945,
 1950,
 1951,
 1952,
 1957,
 1996,
 2100,
 2111,
 2153,
 2163,
 2180,
 2181,
 2237,
 2238,
 2312,
 2313,
 2415,
 2442,
 2696,
 2726,
 2813,
 2814,
 2864,
 2865,
 2931,
 3081,
 3125,
 3132,
 3133,
 3250,
 3304,
 3440,
 3462,
 3519,
 3561,
 3722,
 3745,
 3809,
 3839,
 3927,
 3932,
 3991,
 4012,
 4025,
 4026,
 4159,
 4160,
 4161,
 4363,
 4386,
 4499,
 4531,
 4703,
 4722,
 4723,
 4819,
 4830,
 4831,
 4832,
 4886,
 5031,
 5111,
 5335,
 5421,
 5502,
 5503,
 5661,
 5662,
 5663,
 5737,
 5784,
 5894,
 6123,
 6268,
 6358,
 6544,
 6595,
 6747,
 6838,
 6859,
 6860,
 6955,
 6956,
 7056,
 7172,
 7173,
 7239,
 7272,
 7273,
 7367,
 7401,
 7499,
 7527,
 7601,
 7671,
 7679,
 7772,
 7876,
 7885,
 7930,
 7931,
 7967,
 7968,
 7969,
 8148,
 8163,
 8205,
 8206,
 8207,
 8208,
 8230,
 8241,
 8275,
 8276,
 8320